# TinyML Non-Contact Heart Rate Monitor (rPPG Pipeline)

**Project:** Remote Photoplethysmography (rPPG) on Edge Hardware.
**Author:** [Your Name]

### **Overview**
This notebook implements a computer vision pipeline to detect human heart rate from a standard video feed.
1.  **Ingest:** Reads a video file and extracts the **Green Channel** signal from the forehead (Region of Interest).
2.  **Process:** Applies Digital Signal Processing (DSP) techniques (Butterworth Bandpass Filter) to isolate the pulse.
3.  **Analyze:** Calculates Heart Rate (BPM) using a robust Median Inter-Beat Interval (IBI) algorithm.
4.  **Deploy:** Automatically generates valid **C++ Firmware Code** to run this exact logic on an ESP32 microcontroller.

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
from google.colab import files
import os

# --- Global Configuration ---
FS = 30.0          # Video Frame Rate (Assumed 30fps for standard cameras)
LOW_BPM = 42       # Lowest realistic heart rate to detect
HIGH_BPM = 180     # Highest realistic heart rate to detect
DATA_SLICE = 300   # How many frames to export to C++ (e.g., 300 frames = 10 seconds)

print("✅ Libraries imported and configuration set.")

## **1. Core Processing Functions**
Here we define the logic to extract the signal.
* **Region of Interest (ROI):** We automatically target the center 20% of the frame. This usually captures the forehead or cheek, which are rich in capillary blood flow.
* **Green Channel:** We extract only the Green color channel because hemoglobin absorbs green light most strongly, providing the best Signal-to-Noise Ratio (SNR) for pulse detection.

In [ ]:
def get_roi_rect(frame):
    """Calculates the coordinates of the ROI box, shifted up to the forehead."""
    h, w, _ = frame.shape
    # Box size is 20% of width and height
    box_w, box_h = w // 5, h // 5

    # 1. Calculate the dead-center position first
    center_x, center_y = w // 2, h // 2

    # 2. Calculate standard center box start
    start_x = center_x - (box_w // 2)
    original_start_y = center_y - (box_h // 2)

    # 3. Shift UP by exactly one box height to hit the forehead
    start_y = original_start_y - box_h

    # 4. Safety check: ensure we don't go off the top edge
    if start_y < 0: start_y = 0

    return start_x, start_y, box_w, box_h

def get_roi_avg(frame):
    """Extracts Green Channel average from the target ROI."""
    start_x, start_y, box_w, box_h = get_roi_rect(frame)
    # Slice the image to get only the ROI
    roi = frame[start_y:start_y+box_h, start_x:start_x+box_w]
    # Index 1 is Green in BGR
    return np.mean(roi[:, :, 1])

def process_video_file(filename):
    """Reads video, extracts signal, and grabs CROPPED example frames."""
    cap = cv2.VideoCapture(filename)
    if not cap.isOpened():
        print("Error: Video capture could not open.")
        return np.array([]), []

    raw_values = []
    example_frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Grab Start, Middle, and End frames
    indices_to_grab = [0, frame_count // 2, frame_count - 1]

    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret: break

        # 1. Get Signal Data (uses the new forehead coordinates)
        raw_values.append(get_roi_avg(frame))

        # 2. Grab Cropped Example Frames
        if current_frame in indices_to_grab:
            # Convert BGR to RGB for correct plotting
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Get ROI coordinates
            sx, sy, bw, bh = get_roi_rect(frame)
            # CROP the frame to contain only the ROI
            cropped_roi = rgb_frame[sy:sy+bh, sx:sx+bw]
            example_frames.append(cropped_roi)

        current_frame += 1

    cap.release()

    return np.array(raw_values), example_frames

print("✅ Processing functions updated: Forehead target selected.")

## **2. Data Loading**
This block checks if you have already uploaded a video.
* If a video exists, it uses it (allowing for rapid iteration).
* If no video exists, it prompts you to upload one.
* *Note: Keep videos under 60 seconds for faster processing.*

In [ ]:
# Check for existing video files
video_extensions = ('.mp4', '.mov', '.avi')
existing_files = [f for f in os.listdir('.') if f.lower().endswith(video_extensions)]
filename = ""

if existing_files:
    filename = existing_files[0]
    print(f"✅ Found existing video: '{filename}'")
    print("   (Skipping upload. To use a new video, delete this file from the folder icon on the left.)")
else:
    print("📂 Step 1: Upload your video file.")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
    else:
        print("❌ No file uploaded.")

# Run extraction if file exists
if filename:
    print(f"\nProcessing {filename}...")
    data, example_frames = process_video_file(filename)
    if len(data) == 0:
        print("❌ Error: Could not extract data from video.")
    else:
        print(f"✅ Extracted {len(data)} frames of signal data.")

## **3. Visual Verification**
Before analyzing the signal, we must verify the computer is looking at the correct location. Below are three frames from your video showing the **Region of Interest (Green Box)**.
* *Check:* Ensure the box is on skin (forehead/cheek) and not drifting off-screen.

In [ ]:
if len(example_frames) > 0:
    plt.figure(figsize=(15, 5))
    titles = ['Start Frame', 'Middle Frame', 'End Frame']
    for i, frame in enumerate(example_frames):
        plt.subplot(1, 3, i+1)
        plt.imshow(frame)
        plt.title(titles[i])
        plt.axis('off')
    plt.show()
else:
    print("No frames to display.")

## **4. Digital Signal Processing (DSP)**
Raw video data is noisy. We apply a pipeline to recover the heartbeat:
1.  **Detrending:** Removing the baseline drift caused by breathing or lighting changes.
2.  **Bandpass Filter:** We use a **2nd-order Butterworth filter** to keep only frequencies between **0.7Hz** (42 BPM) and **3.0Hz** (180 BPM).
3.  **Median IBI Calculation:** Instead of averaging beats over time (which is sensitive to silence at the start), we calculate the time interval between every pair of beats and take the median.

In [ ]:
if len(data) > 0:
    # 1. Detrend (Center at 0)
    data = data - np.mean(data)

    # 2. Design Bandpass Filter
    nyquist = 0.5 * FS
    low = (LOW_BPM / 60.0) / nyquist
    high = (HIGH_BPM / 60.0) / nyquist
    b, a = butter(2, [low, high], btype='band')

    # 3. Apply Filter
    filtered_data = filtfilt(b, a, data)

    # 4. Peak Detection
    # We enforce a minimum distance of 0.6 seconds (approx 100 BPM limit) to avoid double-counting noise
    min_dist = int(FS * 0.6)
    peaks, _ = find_peaks(filtered_data, distance=min_dist)

    # 5. Calculate BPM (Median Inter-Beat Interval Method)
    bpm = 0.0
    if len(peaks) > 1:
        intervals = np.diff(peaks) / FS
        bpm_instant = 60.0 / intervals
        bpm = np.median(bpm_instant)

    print(f"✅ DSP Complete. Estimated Heart Rate: {bpm:.2f} BPM")

## **5. Results Visualization**
* **Top Graph (Green):** The raw, noisy signal extracted from the camera. Note the drift and jitter.
* **Bottom Graph (Blue):** The filtered pulse wave. The orange 'x' markers indicate detected heartbeats.

In [ ]:
if len(data) > 0:
    time_axis = np.arange(len(data)) / FS
    plt.figure(figsize=(10, 6))

    # Plot Raw
    plt.subplot(2,1,1)
    plt.plot(time_axis, data, color='green', alpha=0.5, label='Raw Signal')
    plt.legend(loc='upper right')
    plt.title(f"Analysis Result: {bpm:.1f} BPM")
    plt.grid(True, alpha=0.3)

    # Plot Filtered
    plt.subplot(2,1,2)
    plt.plot(time_axis, filtered_data, color='blue', label='Filtered Pulse')
    plt.scatter(time_axis[peaks], filtered_data[peaks], color='orange', marker='x', s=50, label='Detected Beats')
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

## **6. Automated Firmware Generation (MLOps)**
To deploy this to an **ESP32** or **Arduino**, we need C++ code. Manually copying array values is error-prone.
The block below automatically formats our raw Python data and our Filter Coefficients into valid C++ code.

**Instructions:**
1.  Run the cell below.
2.  Copy the output block.
3.  Paste it into the [Wokwi Simulator](https://wokwi.com/projects/new/esp32) to verify the embedded performance.

In [ ]:
if len(data) > 0:
    # Format data array for C++ (Take a slice to save memory)
    cpp_data = data[:DATA_SLICE]
    data_str = ", ".join([f"{x:.2f}" for x in cpp_data])

    cpp_template = f"""#include <Arduino.h>

/* rPPG Heart Rate Filter (Auto-Generated)
   Source Video: {filename}
   Python Estimated BPM: {bpm:.1f}
*/

// --- RAW DATA FROM PYTHON ({len(cpp_data)} frames) ---
const int DATA_LEN = {len(cpp_data)};
float raw_data[] = {{
  {data_str}
}};

// --- FILTER COEFFICIENTS (Butterworth Bandpass) ---
// Generated automatically from Python scipy.signal
float b0 = {b[0]:.6f}, b1 = {b[1]:.6f}, b2 = {b[2]:.6f}, b3 = {b[3]:.6f}, b4 = {b[4]:.6f};
float a0 = {a[0]:.6f}, a1 = {a[1]:.6f}, a2 = {a[2]:.6f}, a3 = {a[3]:.6f}, a4 = {a[4]:.6f};

// History Buffers
float x[5] = {{0,0,0,0,0}};
float y[5] = {{0,0,0,0,0}};

void setup() {{
  Serial.begin(115200);
  Serial.println("Starting Simulation...");
  Serial.println("Raw_Signal,Filtered_Pulse"); // Plotter Legend
}}

void loop() {{
  for(int i=0; i < DATA_LEN; i++) {{

    float input = raw_data[i];

    // Shift History
    for(int k=4; k>0; k--) {{
      x[k] = x[k-1];
      y[k] = y[k-1];
    }}
    x[0] = input;

    // Difference Equation (Direct Form II Transposed / Standard)
    float output = (b0*x[0] + b1*x[1] + b2*x[2] + b3*x[3] + b4*x[4])
                 - (a1*y[1] + a2*y[2] + a3*y[3] + a4*y[4]);

    y[0] = output;

    // Print for Serial Plotter
    Serial.print(input);
    Serial.print(",");
    Serial.println(output * 50);

    delay(33); // Simulate 30 FPS
  }}

  // Stop after one pass
  while(1) {{ delay(10); }}
}}
"""
    print(cpp_template)